# 🎵 HeartMuLa: Music Generation Inference

This notebook allows you to run **HeartMuLa** (Heart Music Language), a state-of-the-art open-source music generation model.

 <a href="https://github.com/HeartMuLa/heartlib">GitHub Repository</a> | <a href="https://arxiv.org/abs/2601.10547">ArXiv Paper</a>

### ⚠️ Runtime Requirement
**GPU is required.** Please ensure you are running on a GPU runtime.
*   **Colab:** Runtime > Change runtime type > T4 GPU (Standard) or A100 (Premium).
*   **Note:** T4 GPUs have limited memory. We use `lazy_load=True` to minimize VRAM usage.

## 1. Setup Environment

In [1]:
# @title Install Dependencies
# @markdown This step clones the repository and installs the required Python packages.

import os
import sys
from IPython.display import clear_output

!nvidia-smi

print("Installing dependencies... (this may take 1-2 minutes)")

# Clone repo
if not os.path.exists("/content/heartlib"):
    %cd /content
    !git clone https://github.com/HeartMuLa/heartlib.git

%cd /content/heartlib

# Install dependencies
!pip install . --quiet
!pip install huggingface_hub --quiet

clear_output()
print("✅ Installation complete.")

✅ Installation complete.


## 2. Download Models
We need to download three components from Hugging Face:
1.  **Configuration & Tokenizer**: `HeartMuLa/HeartMuLaGen`
2.  **Music Model (3B)**: `HeartMuLa/HeartMuLa-oss-3B`
3.  **Audio Codec**: `HeartMuLa/HeartCodec-oss`

In [2]:
# @title Download Checkpoints

import os

# Define paths
ckpt_dir = "/content/heartlib/ckpt"
os.makedirs(ckpt_dir, exist_ok=True)

print("Downloading configuration and tokenizer...")
!huggingface-cli download HeartMuLa/HeartMuLaGen --local-dir {ckpt_dir} --local-dir-use-symlinks False

print("Downloading HeartMuLa-oss-3B model...")
!huggingface-cli download HeartMuLa/HeartMuLa-oss-3B --local-dir {ckpt_dir}/HeartMuLa-oss-3B --local-dir-use-symlinks False

print("Downloading HeartCodec-oss...")
!huggingface-cli download HeartMuLa/HeartCodec-oss --local-dir {ckpt_dir}/HeartCodec-oss --local-dir-use-symlinks False

clear_output()
print("✅ Models downloaded successfully to ./ckpt")

✅ Models downloaded successfully to ./ckpt


## 3. Load Model Pipeline
We initialize the pipeline here. We auto-detect if your GPU supports `bfloat16` (Ampere/A100) or if we should fallback to `float16` / `float32` (T4/Pascal).

In [3]:
import torch
from heartlib import HeartMuLaGenPipeline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available() and torch.cuda.is_bf16_supported():
    mula_dtype = torch.bfloat16
    print("Using bfloat16 precision (Optimal)")
else:
    mula_dtype = torch.float16
    print("Using float16 precision (Fallback for T4/V100)")
codec_dtype = torch.float32
pipe = HeartMuLaGenPipeline.from_pretrained(
    "./ckpt",
    device={"mula": device, "codec": device},
    dtype={"mula": mula_dtype, "codec": codec_dtype},
    version="3B",
    lazy_load=True
)
print("✅ Pipeline loaded successfully!")

Using bfloat16 precision (Optimal)
Model components will be loaded to devices as specified:
  mula: cuda
  codec: cuda
✅ Pipeline loaded successfully!


## 4. Generate Music

Enter your Lyrics and Tags below.
*   **Lyrics**: Structure them with tags like `[Verse]`, `[Chorus]`.
*   **Tags**: Comma-separated genres or moods (e.g., `pop, happy, piano`).
*   **Parameters**:
    *   `Max Duration`: Length of the song in seconds (1 min = 60000ms).
    *   `CFG Scale`: Higher values follow text more closely (1.5 is standard).
    *   `Temperature`: Creativity (1.0 is standard).

In [4]:
# @title 🎹 Inference Settings

lyrics = """[Intro]

[Verse]
The city lights are calling
Through the rain that keeps on falling
A neon glow on wet pavement
A silent wish, a time well spent

[Chorus]
We are running through the night
Chasing shadows, chasing light
In the rhythm of the street
Where the heart and soul both meet

[Outro]
Fading away..."""

tags = "pop, synthwave, emotional, piano, female vocals" # @param {type:"string"}

duration_seconds = 30 # @param {type:"slider", min:10, max:180, step:10}
guidance_scale = 2.1 # @param {type:"slider", min:1.0, max:5.0, step:0.1}
temperature = 1.0 # @param {type:"slider", min:0.1, max:2.0, step:0.1}

# Convert duration to ms
max_audio_length_ms = duration_seconds * 1000

output_path = "./assets/my_generation.mp3"
os.makedirs("./assets", exist_ok=True)

print("Generating music... Please wait.")

with torch.no_grad():
    pipe(
        {
            "lyrics": lyrics,
            "tags": tags,
        },
        max_audio_length_ms=max_audio_length_ms,
        save_path=output_path,
        topk=50,
        temperature=temperature,
        cfg_scale=guidance_scale,
    )

print(f"✅ Generation complete: {output_path}")

Generating music... Please wait.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 375/375 [02:24<00:00,  2.59it/s]


You have set lazy_load=True. Unloading HeartMuLa from device.
CUDA memory before unloading: 12.67 GB
CUDA memory after unloading: 0.01 GB


100%|██████████| 10/10 [00:13<00:00,  1.38s/it]


You have set lazy_load=True. Unloading HeartCodec from device.
CUDA memory before unloading: 6.20 GB
CUDA memory after unloading: 0.01 GB
✅ Generation complete: ./assets/my_generation.mp3


In [5]:
# @title 🎧 Listen to the Song
from IPython.display import Audio, display

if os.path.exists(output_path):
    display(Audio(output_path))
else:
    print("Audio file not found. Please run the generation cell first.")

In [6]:
# @title (Optional) Download MP3
# @markdown Run this to download the generated file to your local computer.
from google.colab import files
if os.path.exists(output_path):
    files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>